In [7]:
###############################################################
import os
import cv2
import numpy as np
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from skimage import exposure

# Function to load and extract HOG features from images in a directory and create labels
def load_and_extract_hog_features(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            # Resize the image if needed
            img = cv2.resize(img, (100, 100))
            
            # Extract HOG features
            features, hog_image = hog(img, orientations=8, pixels_per_cell=(8, 8),
                                      cells_per_block=(2, 2), visualize=True)
            
            # Enhance the contrast of the HOG image
            hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
            
            images.append(features)
            labels.append(label)
    return images, labels

# Load and extract HOG features for open eyes dataset
open_folder = "../../Desktop/drowniess detection project/eye dataset/train/Open_Eyes"
open_images, open_labels = load_and_extract_hog_features(open_folder, label=1)

# Load and extract HOG features for closed eyes dataset
closed_folder = "../../Desktop/drowniess detection project/eye dataset/train/Closed_Eyes"
closed_images, closed_labels = load_and_extract_hog_features(closed_folder, label=0)

# Combine the datasets
X = np.vstack([open_images, closed_images])
y = np.concatenate([open_labels, closed_labels])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the SVM model
model = SVC(kernel='linear')

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Save the trained model to a file using joblib
joblib.dump(model, 'svm_eye_model_hog.joblib')

# Test the model on a live video stream
cap = cv2.VideoCapture(0)  # Use 0 for the default camera

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to capture frame.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Resize the image if needed
    img = cv2.resize(gray, (100, 100))
    
    # Extract HOG features for the entire image
    features = hog(img, orientations=8, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2))
    
    # Reshape the features to match the shape used during training
    features = features.reshape(1, -1)

    # Make a prediction for the eyes
    prediction = model.predict(features)

    # Display the result on the video stream
    if prediction == 1:
        cv2.putText(frame, 'Open Eyes', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    else:
        cv2.putText(frame, 'Closed Eyes', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('Eye Detection', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


FileNotFoundError: [WinError 3] The system cannot find the path specified: '../../Desktop/drowniess detection project/eye dataset/train/Open_Eyes'